In [1]:
import os
import csv
import cv2
import tensorflow as tf
import numpy as np

In [2]:
print(os.getcwd())

c:\Users\User\Desktop\sophomore_2nd_semester\Artificial_Intelligence\final_project\Human-pose-estimation\dataset


In [3]:
title = ['file_name','nose_x','nose_y','nose_score','left_eye_x','left_eye_y',
                     'left_eye_score','right_eye_x','right_eye_y','right_eye_score',
                     'left_ear_x','left_ear_y','left_ear_score','right_ear_x','right_ear_y'
                     ,'right_ear_score','left_shoulder_x','left_shoulder_y','left_shoulder_score'
                     ,'right_shoulder_x','right_shoulder_y','right_shoulder_score','left_elbow_x'
                     ,'left_elbow_y','left_elbow_score','right_elbow_x','right_elbow_y',
                     'right_elbow_score','left_wrist_x','left_wrist_y','left_wrist_score',
                     'right_wrist_x','right_wrist_y','right_wrist_score','left_hip_x','left_hip_y',
                     'left_hip_score','right_hip_x','right_hip_y','right_hip_score','left_knee_x',
                     'left_knee_y','left_knee_score','right_knee_x','right_knee_y','right_knee_score', 
                     'left_ankle_x','left_ankle_y','left_ankle_score','right_ankle_x','right_ankle_y',
                     'right_ankle_score','class_no','class_name']

In [4]:
from movenet import Movenet
movenet = Movenet('movenet_thunder')
print(movenet)

def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.
 
  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.
 
  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape
 
  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)
 
  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person

In [5]:
data_path = os.getcwd()+r'\test_data'
poses = os.listdir(data_path)
output_folder = 'test_data2.csv'
with open(output_folder, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(title)
    pose_index = 0
    for pose in poses:
        input_folder = os.path.join(data_path, pose)
        imageslist = os.listdir(input_folder)
        for image_path in imageslist:
            image = tf.io.read_file(os.path.join(input_folder, image_path))
            image = tf.io.decode_jpeg(image)
            person = detect(image)
            pose_landmarks = np.array([[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints])
            coordinates = pose_landmarks.flatten().astype(str).tolist()
            writer.writerow([image_path] + coordinates+ [pose_index] + [pose])
        pose_index+=1

In [ ]:
data_path = os.getcwd()+r'\train_data'
poses = os.listdir(data_path)
train_output_folder = 'train_data2.csv'
valid_output_folder = 'valid_data2.csv'
train_writer = csv.writer(open(train_output_folder, 'w', newline=''))
valid_writer = csv.writer(open(valid_output_folder, 'w', newline=''))
train_writer.writerow(title)
valid_writer.writerow(title)

valid_rate = 0.2

pose_index = 0
for pose in poses:
    input_folder = os.path.join(data_path, pose)
    imageslist = os.listdir(input_folder)
    
    data_size = len(imageslist)
    for index, image_path in enumerate(imageslist):
        
        image = tf.io.read_file(os.path.join(input_folder, image_path))
        image = tf.io.decode_jpeg(image)
        person = detect(image)
        pose_landmarks = np.array([[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
            for keypoint in person.keypoints])
        coordinates = pose_landmarks.flatten().astype(str).tolist()

        if(index<=data_size*valid_rate):
            valid_writer.writerow([image_path] + coordinates+ [pose_index] + [pose])
        else:
            train_writer.writerow([image_path] + coordinates+ [pose_index] + [pose])

    pose_index+=1